# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, concat, count, desc, explode, lit, min, max, split, stddev, udf
from pyspark.sql.types import IntegerType

from pyspark.ml.feature import RegexTokenizer, VectorAssembler, Normalizer, StandardScaler, \
    RegexTokenizer, CountVectorizer,IDF, StringIndexer, PCA, StopWordsRemover
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import re

In [2]:
# create a Spark session
spark = SparkSession.builder \
    .master("local") \
    .appName("sparkify") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [6]:
stack_overflow_data = '../spark_project_data/mini_sparkify_event_data.json'
df = spark.read.json(stack_overflow_data)
df.persist()

DataFrame[artist: string, auth: string, firstName: string, gender: string, itemInSession: bigint, lastName: string, length: double, level: string, location: string, method: string, page: string, registration: bigint, sessionId: bigint, song: string, status: bigint, ts: bigint, userAgent: string, userId: string]

In [7]:
df.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')

In [8]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [9]:
df.describe().show()

+-------+------------------+----------+---------+------+------------------+--------+-----------------+------+-----------------+------+-------+--------------------+-----------------+--------------------+------------------+--------------------+--------------------+-----------------+
|summary|            artist|      auth|firstName|gender|     itemInSession|lastName|           length| level|         location|method|   page|        registration|        sessionId|                song|            status|                  ts|           userAgent|           userId|
+-------+------------------+----------+---------+------+------------------+--------+-----------------+------+-----------------+------+-------+--------------------+-----------------+--------------------+------------------+--------------------+--------------------+-----------------+
|  count|            228108|    286500|   278154|278154|            286500|  278154|           228108|286500|           278154|286500| 286500|            

In [13]:
df.count()

286500

In [71]:
def show_value_counts(dataframe):
    """
    Prints out null_values_counts and distinct_values_counts of the dataframe
    
    Input: df - a spark dataframe
    
    Output:
            print(column_name)
            print(null_value_counts and %) 
            print(distinct_value_counts)
            print(distinct_values) if less than 100
            
    """
    
    color_counts = lambda x: '\033[1m'+'\033[91m'+str(x)+'\033[0m' if x>0 else '\033[1m'+'\033[92m'+str(x)+'\033[0m'
    
    for col in df.columns:
        print('\033[1m'+'\033[95m'+col+'\033[0m','-'*(100-len(col)))
        null_value_counts = df.filter(df[col].isNull()).count()
        distinct_values = df.select(col).distinct().sort(col).collect()
        distinct_value_counts = len(distinct_values)
        print('Null Value Count: {} / {}%'.format(color_counts(null_value_counts),round(null_value_counts/df.count()*100,2)))
        print('Unique Value Count:', distinct_value_counts)
        if distinct_value_counts < 100:
            print('Unique Values:', distinct_values)
        print()

In [72]:
show_value_counts(df)

artist ----------------------------------------------------------------------------------------------
Null Value Count: 58392 / 20.38%
Unique Value Count: 17656

auth ------------------------------------------------------------------------------------------------
Null Value Count: 0 / 0.0%
Unique Value Count: 4
Unique Values: [Row(auth='Cancelled'), Row(auth='Guest'), Row(auth='Logged In'), Row(auth='Logged Out')]

firstName -------------------------------------------------------------------------------------------
Null Value Count: 8346 / 2.91%
Unique Value Count: 190

gender ----------------------------------------------------------------------------------------------
Null Value Count: 8346 / 2.91%
Unique Value Count: 3
Unique Values: [Row(gender=None), Row(gender='F'), Row(gender='M')]

itemInSession ---------------------------------------------------------------------------------------
Null Value Count: 0 / 0.0%
Unique Value Count: 1322

lastName -----------------------------------

Null Value Count: 0 / 0.0%
Unique Value Count: 226



In [43]:
col='userAgent'
distinct_values = df.select(col).distinct().sort(col).collect()
for i in distinct_values:
    print(i)

Row(userAgent=None)
Row(userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10) AppleWebKit/600.1.3 (KHTML, like Gecko) Version/8.0 Safari/600.1.3"')
Row(userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10) AppleWebKit/600.1.8 (KHTML, like Gecko) Version/8.0 Safari/600.1.8"')
Row(userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
Row(userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"')
Row(userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_5) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/6.1.5 Safari/537.77.4"')
Row(userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
Row(userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36"')
Row(userAgent='"Mozi

In [87]:
col='userId'
df.select(col).distinct().sort(col).collect()

[Row(userId=''),
 Row(userId='10'),
 Row(userId='100'),
 Row(userId='100001'),
 Row(userId='100002'),
 Row(userId='100003'),
 Row(userId='100004'),
 Row(userId='100005'),
 Row(userId='100006'),
 Row(userId='100007'),
 Row(userId='100008'),
 Row(userId='100009'),
 Row(userId='100010'),
 Row(userId='100011'),
 Row(userId='100012'),
 Row(userId='100013'),
 Row(userId='100014'),
 Row(userId='100015'),
 Row(userId='100016'),
 Row(userId='100017'),
 Row(userId='100018'),
 Row(userId='100019'),
 Row(userId='100021'),
 Row(userId='100022'),
 Row(userId='100023'),
 Row(userId='100024'),
 Row(userId='100025'),
 Row(userId='101'),
 Row(userId='102'),
 Row(userId='103'),
 Row(userId='104'),
 Row(userId='105'),
 Row(userId='106'),
 Row(userId='107'),
 Row(userId='108'),
 Row(userId='109'),
 Row(userId='11'),
 Row(userId='110'),
 Row(userId='111'),
 Row(userId='112'),
 Row(userId='113'),
 Row(userId='114'),
 Row(userId='115'),
 Row(userId='116'),
 Row(userId='117'),
 Row(userId='118'),
 Row(userId='

In [78]:
pd_df = df.filter((df.firstName.isNull())).toPandas()

In [82]:
pd_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged Out,None,None,100,None,None,free,None,GET,Home,None,8,None,200,1538355745000,None,
1,None,Logged Out,None,None,101,None,None,free,None,GET,Help,None,8,None,200,1538355807000,None,
2,None,Logged Out,None,None,102,None,None,free,None,GET,Home,None,8,None,200,1538355841000,None,
3,None,Logged Out,None,None,103,None,None,free,None,PUT,Login,None,8,None,307,1538355842000,None,
4,None,Logged Out,None,None,2,None,None,free,None,GET,Home,None,240,None,200,1538356678000,None,


In [86]:
pd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8346 entries, 0 to 8345
Data columns (total 18 columns):
artist           0 non-null object
auth             8346 non-null object
firstName        0 non-null object
gender           0 non-null object
itemInSession    8346 non-null int64
lastName         0 non-null object
length           0 non-null object
level            8346 non-null object
location         0 non-null object
method           8346 non-null object
page             8346 non-null object
registration     0 non-null object
sessionId        8346 non-null int64
song             0 non-null object
status           8346 non-null int64
ts               8346 non-null int64
userAgent        0 non-null object
userId           8346 non-null object
dtypes: int64(4), object(14)
memory usage: 1.1+ MB


# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.